### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7fb73420e680>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7fb73420e710>
layer: Linear
func: <function parse_linear_layer at 0x7fb73420e8c0>
layer: Softmax
func: <function parse_activation_layer at 0x7fb73420e830>
layer: ReLU
func: <function parse_activation_layer at 0x7fb73420e830>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7fb73420e7a0>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7fb73420e7a0>
handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
layer: NodeBlock
func: <function parse_NodeBlock at 0x7fb73421bef0>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7fb7342200e0>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7fb734220290>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7fb734220440>
layer: NodeEncoder
func: <f

### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40)
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)

for name, submodule in torch_model.named_modules():
    if name != "":
        print(f"{name}: {submodule}")

R1: RelationalModel(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
R1.layers: Sequential(
  (0): Linear(in_features=15, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=5, bias=True)
)
R1.layers.0: Linear(in_features=15, out_features=40, bias=True)
R1.layers.1: ReLU()
R1.layers.2: Linear(in_features=40, out_features=40, bias=True)
R1.layers.3: ReLU()
R1.layers.4: Linear(in_features=40, out_features=5, bias=True)
O: ObjectModel(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
O.layers: Sequential(
  (

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
#         "node_dim": 3,
        "node_attr": 3,
        "node_dim": common_dim,
#         "edge_dim": 4,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<64,17>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<64,17>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
edge encoder b4 update_dict["last_edge_update"]: edge_attr
edge encoder after update_dict["last_edge_update"]: layer5_out
EdgeBlock update_dict["last_edge_update"] b4: layer5_out
EdgeBlock update_dict["last_edge_update"] after: layer6_out
type(update_dict): <class 'dict'>
nodeblock layer_dict: {'name': 'O', 'n_node': 28, 'n_edge': 37, 'node_dim': 5, 'edge_dim': 5, 'activate_final': 'false', 'activation': 'linear', 'n_layers': 3, 'out_dim': 5, 'class_name': 'NodeBlock', 'inputs': ['layer4_out', 'layer7_out'], 'outputs': ['layer8_out']}
EdgeBlock update_dict["last_edge_update"] b4: layer6_out
EdgeBlock update_dict["last_edge_update"] after: layer9_out
NodeEncoder name: node_encoder
self

In [7]:
hls_model.compile()

Writing HLS project
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb7110805d0>])
layer name: node_attr. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711080910>])
layer name: edge_attr. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711080890>])
layer name: edge_index. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711017d50>])
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
layer name: node_encoder. func: ['nnet::encoder<input_t, layer4_t, config4>(node_attr, layer4_out, w4, b4);']
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711017810>])
def_cpp: layer5_t layer5_out[N_LAYER_1_5*N_LAYER_2_5]
layer name: edge_encoder. func: ['nnet::encoder<input2_t, layer5_t, config5>(edge_attr, layer5_out, w5, b5);']
layer.get_variables(): odict

Done
lib_name: firmware/myproject-Ad43B2B3.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [19]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [20]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

print((np.abs(torch_pred- hls_pred)))
print(torch_pred)
print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
Node Encoder output mean: 0.2295726239681244. std: 0.38199612498283386
Edge Encoder output mean: 0.15614545345306396. std:0.19126424193382263
R1 output mean: -0.007930385880172253, std: 0.10171911865472794
O output mean: 0.003440827364102006, std: 0.07398408651351929
R2 output mean: -0.07871315628290176, std: 0.0004216776869725436
torch pred shape: (37,)
hls_pred.shape: (37,)
MSE: 2.95171371789138e-08
[2.4983287e-04 2.2977591e-04 2.4873018e-04 2.3421645e-04 2.2894144e-04
 2.1168590e-04 2.2783875e-04 2.1636486e-04 2.0387769e-04 1.8793344e-04
 2.0277500e-04 1.9210577e-04 1.0699034e-05 1.0102987e-05 1.4990568e-05
 6.3478947e-06 4.5269728e-05 7.3015690e-06 1.1444092e-05 5.4836273e-06
 1.5050173e-05 4.7147274e-05 1.0046363e-04 1.0386109e-04 9.5069408e-05
 1.0526180e-04 2.7269125e-05 1.7791986e-04 1.8274784e-04 1.7195940e-04
 1.8429756e-04 1.2406707e-04 2.6315451e-04 2.6923418e-04 2.5627017e-04
 2.7239323e-04 1.9186735e-04]
[0.48

In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [21]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

Node Encoder output mean: 0.2295726239681244. std: 0.38199612498283386
Edge Encoder output mean: 0.15614545345306396. std:0.19126424193382263
R1 output mean: -0.007930385880172253, std: 0.10171911865472794
O output mean: 0.003440827364102006, std: 0.07398408651351929
R2 output mean: -0.07871315628290176, std: 0.0004216776869725436
MSE: 2.95171371789138e-08
Node Encoder output mean: 0.18455061316490173. std: 0.3637453317642212
Edge Encoder output mean: 0.1262378692626953. std:0.1998356133699417
R1 output mean: -0.01077501755207777, std: 0.10436180979013443
O output mean: -0.013840502128005028, std: 0.0823395699262619
R2 output mean: -0.08093857020139694, std: 0.0013533675810322165
MSE: 5.046616934123449e-06
Node Encoder output mean: 0.1555982381105423. std: 0.3277280926704407
Edge Encoder output mean: 0.12424665689468384. std:0.19797925651073456
R1 output mean: -0.011131918989121914, std: 0.10759347677230835
O output mean: -0.017698654904961586, std: 0.06976039707660675
R2 output mean: 

In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [13]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =1000):
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [14]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7fb710fedcd0>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7fb710ffcd90>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7fb711001bd0>}


In [29]:
from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
node_scaler = StandardScaler()
edge_scaler = StandardScaler()
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

    for data in graphs: # fit data to scaler b4 feedforwarding
#         print(f"data type: {type(data)}")
        node_scaler.fit(data.x) 
        edge_scaler.fit(data.edge_attr) 
    
    for data in graphs:
       
        data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
        data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data = [data.x, edge_attr, edge_index]
        data.hls_data[0] = node_scaler.transform(data.hls_data[0])
        data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
        hls_pred = hls_model.predict(data.hls_data)
        MSE = mean_squared_error(torch_pred, hls_pred)
        print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 1000
writing test bench data for 1st graph
Node Encoder output mean: 0.21285095810890198. std: 0.46618175506591797
Edge Encoder output mean: 0.08031339943408966. std:0.32578471302986145
R1 output mean: -0.010169941000640392, std: 0.12059379369020462
O output mean: -0.009237919934093952, std: 0.07782881706953049
R2 output mean: -0.08048208057880402, std: 0.0020673905964940786
MSE: 5.02540524394135e-06
Node Encoder output mean: 0.16056491434574127. std: 0.4104364216327667
Edge Encoder output mean: 0.08566033095121384. std:1.2433719635009766
R1 output mean: -0.0058358218520879745, std: 0.13089333474636078
O output mean: -0.007866729982197285, std: 0.06641782075166702
R2 output mean: -0.08171983063220978, std: 0.003995540551841259
MSE: 3.828462922683684e-06
Node Encoder output mean: 0.20347192883491516. std: 0.47317934036254883
Edge Encoder output mean: 0.10707024484872818. std:0.3411283493041992
R1 output mean: -0.007830968126654625, std: 0.11429852992296219
O output mean: -0.00

R1 output mean: -0.004458477254956961, std: 0.1247839629650116
O output mean: -0.007643177639693022, std: 0.0629432275891304
R2 output mean: -0.08081430196762085, std: 0.003111589001491666
MSE: 1.0241641348329722e-06
Node Encoder output mean: 0.1597433239221573. std: 0.49484169483184814
Edge Encoder output mean: 0.14810937643051147. std:1.2498884201049805
R1 output mean: -0.006582790520042181, std: 0.12424812465906143
O output mean: -0.009193512611091137, std: 0.07960272580385208
R2 output mean: -0.08111162483692169, std: 0.00399339385330677
MSE: 1.0768475476652384e-05
Node Encoder output mean: 0.2192920297384262. std: 0.3745594918727875
Edge Encoder output mean: -0.0021347280126065016. std:1.097256064414978
R1 output mean: -0.011230512522161007, std: 0.1286168098449707
O output mean: -0.0054777548648417, std: 0.0659862607717514
R2 output mean: -0.08097422868013382, std: 0.0028265377040952444
MSE: 3.50994014297612e-06
Node Encoder output mean: 0.13934096693992615. std: 0.40903469920158

MSE: 3.878878487739712e-06
Node Encoder output mean: 0.12844419479370117. std: 0.5031536817550659
Edge Encoder output mean: 0.07234877347946167. std:0.46228519082069397
R1 output mean: -0.01869475096464157, std: 0.12169618159532547
O output mean: -0.01453562080860138, std: 0.07847744971513748
R2 output mean: -0.08520638942718506, std: 0.001273695263080299
MSE: 2.2281508790911175e-05
Node Encoder output mean: 0.18005062639713287. std: 0.39051735401153564
Edge Encoder output mean: 0.08566033840179443. std:1.0858949422836304
R1 output mean: -0.00789953675121069, std: 0.12694516777992249
O output mean: -0.007612098939716816, std: 0.06426180899143219
R2 output mean: -0.08097955584526062, std: 0.003887366270646453
MSE: 3.1196143481793115e-06
Node Encoder output mean: 0.19005170464515686. std: 0.45559003949165344
Edge Encoder output mean: 0.08627094328403473. std:1.4853429794311523
R1 output mean: -0.004669394809752703, std: 0.13208411633968353
O output mean: -0.008329636417329311, std: 0.076

MSE: 8.51022923598066e-06
Node Encoder output mean: 0.23764225840568542. std: 0.43116140365600586
Edge Encoder output mean: 0.05088835954666138. std:0.38722556829452515
R1 output mean: -0.012287331745028496, std: 0.12030626833438873
O output mean: -0.004403323400765657, std: 0.07077083736658096
R2 output mean: -0.07974979281425476, std: 0.0014636088162660599
MSE: 3.148855512336013e-06
Node Encoder output mean: 0.1134570837020874. std: 0.37867963314056396
Edge Encoder output mean: 0.08561524748802185. std:0.32078710198402405
R1 output mean: -0.005653398111462593, std: 0.11186475306749344
O output mean: -0.008262265473604202, std: 0.061041347682476044
R2 output mean: -0.08478353172540665, std: 0.0010973353637382388
MSE: 1.2129131391702686e-05
Node Encoder output mean: 0.15900470316410065. std: 0.4848349392414093
Edge Encoder output mean: 0.06053512543439865. std:1.2187687158584595
R1 output mean: -0.01371237263083458, std: 0.12279759347438812
O output mean: -0.009526596404612064, std: 0.

MSE: 9.345529178972356e-06
Node Encoder output mean: 0.1428818553686142. std: 0.49559345841407776
Edge Encoder output mean: 0.07651812583208084. std:0.4423303008079529
R1 output mean: -0.005004022270441055, std: 0.11694662272930145
O output mean: -0.0063441721722483635, std: 0.0735703855752945
R2 output mean: -0.08401092141866684, std: 0.000897421152330935
MSE: 1.097909171221545e-05
Node Encoder output mean: 0.1793263554573059. std: 0.35732027888298035
Edge Encoder output mean: 0.08566032350063324. std:1.2249348163604736
R1 output mean: -0.0062704989686608315, std: 0.1251298189163208
O output mean: -0.007968748919665813, std: 0.06777666509151459
R2 output mean: -0.08009401708841324, std: 0.004100659862160683
MSE: 6.480005595221883e-06
Node Encoder output mean: 0.12752513587474823. std: 0.5324480533599854
Edge Encoder output mean: 0.08989952504634857. std:0.4390154778957367
R1 output mean: -0.0073679061606526375, std: 0.12262777984142303
O output mean: -0.008955078199505806, std: 0.0699

...

MSE: 6.98345775163034e-06
Node Encoder output mean: 0.10667786002159119. std: 0.6056032776832581
Edge Encoder output mean: 0.16671033203601837. std:0.6177224516868591
R1 output mean: -0.008502691052854061, std: 0.1195729672908783
O output mean: -0.0114411236718297, std: 0.07909821718931198
R2 output mean: -0.084917351603508, std: 0.0027078413404524326
MSE: 1.4301911505754106e-05
Node Encoder output mean: 0.11253971606492996. std: 0.41588690876960754
Edge Encoder output mean: 0.16357259452342987. std:4.270092964172363
R1 output mean: -0.0055595687590539455, std: 0.20330071449279785
O output mean: -0.014239397831261158, std: 0.07044798880815506
R2 output mean: -0.08214342594146729, std: 0.00846564769744873
MSE: 1.317680835200008e-05
Node Encoder output mean: 0.18357709050178528. std: 0.5018863081932068
Edge Encoder output mean: 0.1190401241183281. std:3.639159917831421
R1 output mean: -0.012916401028633118, std: 0.19414159655570984
O output mean: -0.010237344540655613, std: 0.07633610814

MSE: 3.17157173412852e-05
Node Encoder output mean: 0.09012248367071152. std: 0.5499573349952698
Edge Encoder output mean: 0.1331567019224167. std:0.45185208320617676
R1 output mean: -0.010901977308094501, std: 0.12173867225646973
O output mean: -0.01126060076057911, std: 0.07553589344024658
R2 output mean: -0.08408530801534653, std: 0.0021137825679033995
MSE: 1.6289834093186073e-05
Node Encoder output mean: 0.22272665798664093. std: 0.517616331577301
Edge Encoder output mean: 0.26778721809387207. std:0.516043484210968
R1 output mean: -0.0072220745496451855, std: 0.12455488741397858
O output mean: -0.010333515703678131, std: 0.08155812323093414
R2 output mean: -0.07876553386449814, std: 0.0011263128835707903
MSE: 1.2702508911388577e-06
Node Encoder output mean: 0.12426116317510605. std: 0.43659016489982605
Edge Encoder output mean: 0.16357262432575226. std:4.512545108795166
R1 output mean: -0.00907263346016407, std: 0.21860085427761078
O output mean: -0.012163079343736172, std: 0.06759

MSE: 5.768991741206264e-06
Node Encoder output mean: 0.154914990067482. std: 0.5120732188224792
Edge Encoder output mean: 0.18130838871002197. std:4.656052589416504
R1 output mean: -0.008223488926887512, std: 0.22240649163722992
O output mean: -0.013134108856320381, std: 0.082646943628788
R2 output mean: -0.0815945416688919, std: 0.00916687585413456
MSE: 1.2589016478159465e-05
Node Encoder output mean: 0.1716824173927307. std: 0.4744793772697449
Edge Encoder output mean: 0.16357260942459106. std:3.9827940464019775
R1 output mean: -0.011311627924442291, std: 0.20019875466823578
O output mean: -0.011471729725599289, std: 0.07551142573356628
R2 output mean: -0.08097221702337265, std: 0.007757231593132019
MSE: 6.976484655751847e-06
Node Encoder output mean: 0.05050020292401314. std: 0.581360399723053
Edge Encoder output mean: 0.2246408611536026. std:3.0737009048461914
R1 output mean: -0.011760862544178963, std: 0.17063137888908386
O output mean: -0.013105539605021477, std: 0.07657717168331

MSE: 1.0960609870380722e-05
Node Encoder output mean: 0.07032129168510437. std: 0.5664312243461609
Edge Encoder output mean: 0.18946920335292816. std:0.5545573234558105
R1 output mean: -0.016395913437008858, std: 0.11450108140707016
O output mean: -0.015390702523291111, std: 0.07477013021707535
R2 output mean: -0.08380984514951706, std: 0.0011073843343183398
MSE: 2.490379301889334e-05
Node Encoder output mean: 0.14075210690498352. std: 0.6149600148200989
Edge Encoder output mean: 0.1319953352212906. std:0.6671621203422546
R1 output mean: -0.009059182368218899, std: 0.12398781627416611
O output mean: -0.009799671359360218, std: 0.0811409130692482
R2 output mean: -0.08237259089946747, std: 0.0034322005230933428
MSE: 5.95135770709021e-06
Node Encoder output mean: 0.11203516274690628. std: 0.661571741104126
Edge Encoder output mean: 0.1790315955877304. std:0.6649762392044067
R1 output mean: -0.01216039527207613, std: 0.13052618503570557
O output mean: -0.012448285706341267, std: 0.07716494

MSE: 1.6841018805280328e-05
Node Encoder output mean: 0.10118651390075684. std: 0.5885118246078491
Edge Encoder output mean: 0.1528613418340683. std:0.4008869230747223
R1 output mean: -0.0033854935318231583, std: 0.12209528684616089
O output mean: -0.010342612862586975, std: 0.073736771941185
R2 output mean: -0.0843205526471138, std: 0.002298532985150814
MSE: 1.0753408787422813e-05
Node Encoder output mean: 0.18535076081752777. std: 0.49742835760116577
Edge Encoder output mean: 0.1174251064658165. std:2.7917656898498535
R1 output mean: -0.01355176791548729, std: 0.16309967637062073
O output mean: -0.011431659571826458, std: 0.07903562486171722
R2 output mean: -0.07975832372903824, std: 0.005827056709676981
MSE: 8.298077773361001e-06
Node Encoder output mean: 0.12251581251621246. std: 0.5526598691940308
Edge Encoder output mean: 0.17390844225883484. std:1.8064274787902832
R1 output mean: -0.017143117263913155, std: 0.13561701774597168
O output mean: -0.012819408439099789, std: 0.0815672

In [16]:
MSE_l.shape

(3000,)

In [17]:
torch_model.state_dict()

OrderedDict([('R1.layers.0.weight',
              tensor([[-0.2538,  0.1386, -0.2459,  0.0432, -0.1325, -0.1588, -0.2042,  0.0966,
                        0.1674, -0.1823,  0.1546, -0.0666, -0.0163,  0.0235, -0.0862],
                      [-0.0900,  0.1414,  0.1040, -0.2356,  0.0520,  0.2389, -0.2221, -0.1938,
                       -0.0350, -0.1863, -0.1852, -0.2055,  0.0357,  0.2060, -0.0510],
                      [-0.0391,  0.0447,  0.1168, -0.1439, -0.2567, -0.1247, -0.2079, -0.1791,
                        0.1861, -0.2454, -0.0700,  0.0754, -0.1143, -0.0855,  0.1400],
                      [-0.1776,  0.0483,  0.0781,  0.1151,  0.2524, -0.2195, -0.0683, -0.1887,
                        0.2208,  0.1440, -0.2153,  0.1516, -0.2304, -0.0295,  0.2118],
                      [ 0.2282, -0.1460,  0.0092,  0.1550, -0.2500, -0.1213, -0.0713, -0.1675,
                       -0.1598, -0.1473,  0.0241,  0.2531,  0.1716,  0.1419, -0.0643],
                      [-0.0337,  0.0474,  0.2516,  0.0